In [1]:
import os
import shutil

# Define paths for the source folders and destination folder
source_folder1 = r"D:\Major Project\Final Proper\data_need_to_split\Drowsy"
source_folder2 = r"D:\Major Project\Final Proper\data_need_to_split\NonDrowsy"
destination_folder = r"D:\Major Project\Final Proper\data_need_to_split\American"

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Initialize a counter for renaming images
counter = 1

# Function to move and rename images
def move_and_rename_images(source_folder, destination_folder, counter):
    for filename in os.listdir(source_folder):
        # Get the file extension
        file_extension = os.path.splitext(filename)[1]
        
        # Check if the file is an image (e.g., .jpg, .png)
        if file_extension.lower() in ['.jpg', '.jpeg', '.png', '.gif', '.bmp']:
            # Create a new filename with sequential numbering
            new_filename = f"{counter}{file_extension}"
            
            # Define source and destination paths
            source_path = os.path.join(source_folder, filename)
            destination_path = os.path.join(destination_folder, new_filename)
            
            # Move and rename the image
            shutil.move(source_path, destination_path)
            
            # Increment the counter
            counter += 1
    return counter

In [2]:
# Process both source folders
counter = move_and_rename_images(source_folder1, destination_folder, counter)
counter = move_and_rename_images(source_folder2, destination_folder, counter)

print("Images moved and renamed successfully!")

Images moved and renamed successfully!


In [1]:
import socket
import struct
import zlib
import tempfile
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report
# from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras import layers, initializers, backend as K
from sklearn.metrics import confusion_matrix, accuracy_score
from cryptography.fernet import Fernet
import base64
import hashlib

In [2]:
from tensorflow.keras import layers, initializers, backend as K

# **Custom Capsule Layers**
class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1) + K.epsilon())

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]
        self.W = self.add_weight(
            shape=[1, self.input_num_capsule, self.num_capsule, self.dim_capsule, self.input_dim_capsule],
            initializer=initializers.glorot_uniform(),
            name='W'
        )
        self.built = True

    def call(self, inputs):
        inputs_expand = K.expand_dims(K.expand_dims(inputs, 2), 2)
        W_tiled = K.tile(self.W, [K.shape(inputs)[0], 1, 1, 1, 1])
        inputs_hat = tf.squeeze(tf.matmul(W_tiled, inputs_expand, transpose_b=True), axis=-1)
        b = tf.zeros(shape=[K.shape(inputs)[0], self.input_num_capsule, self.num_capsule])

        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            outputs = tf.reduce_sum(inputs_hat * K.expand_dims(c, -1), axis=1)
            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat * K.expand_dims(c, -1), axis=-1)
        return outputs

# **Build MobileNet + Capsule Model**
class MobileNetCapsNet:
    def __init__(self, input_shape=(224, 224, 3)):
        self.input_shape = input_shape
        self.model = self._build_model()

    def _build_model(self):
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=self.input_shape,
            include_top=False,
            weights='imagenet'
        )
        base_model.trainable = False  # Freeze MobileNet

        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Reshape((-1, 64))(x)
        x = CapsuleLayer(num_capsule=2, dim_capsule=8, routings=1)(x)
        outputs = Length()(x)

        return tf.keras.Model(inputs=base_model.input, outputs=outputs)

# **Margin Loss Function**
def margin_loss(y_true, y_pred):
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=2)
    L = y_true * tf.square(tf.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * tf.square(tf.maximum(0., y_pred - 0.1))
    return tf.reduce_mean(tf.reduce_sum(L, axis=1))

In [3]:
# custom_objects = {
#     'CapsuleLayer': CapsuleLayer,
#     'PrimaryCap': PrimaryCap,
#     'squash': squash,
#     'capsule_length': capsule_length,
#     'margin_loss': margin_loss,  # include if you're using a custom margin loss

# }
model=tf.keras.models.load_model(r"D:\Major Project\Final Proper\drowsiness_lesser_india_plus_eye.keras",custom_objects={"CapsuleLayer": CapsuleLayer,"Length":Length,"margin_loss":margin_loss})

In [4]:
print(model.output_shape)


(None, 4)


In [6]:
import os
import cv2
import numpy as np
import tensorflow as tf
import shutil
import mediapipe as mp

# **Load MediaPipe Face Mesh**
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

# **Load Pretrained Model**
model = model  # Ensure the path is correct
print("✅ Model loaded successfully!")

# **Get Input Folder from User**
input_folder = r"D:\Major Project\final datset\train\drowsy"  # Update the input directory

# **Check if the directory exists**
if not os.path.exists(input_folder):
    print(f"❌ Error: The directory '{input_folder}' does not exist.")
    exit()

# **Define Output Folders for Each Class**
output_base = os.path.join(input_folder, "Dataset_Classified")
class_folders = {
    "Eyes Closed": os.path.join(output_base, "Eyes_Closed"),
    "Happy": os.path.join(output_base, "Happy"),
    "Neutral": os.path.join(output_base, "Neutral"),
    "Yawning": os.path.join(output_base, "Yawning"),
    "Uncertain": os.path.join(output_base, "Uncertain")
}

# **Create Output Folders if They Don't Exist**
for folder in class_folders.values():
    os.makedirs(folder, exist_ok=True)

# **Class Labels Mapping**
CLASS_LABELS = {
    0: "Eyes Closed",
    1: "Happy",
    2: "Neutral",
    3: "Yawning"
}

# **Preprocessing Function**
def preprocess_image(img, target_size=(224, 224)):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize
    return np.expand_dims(img, axis=0)  # Add batch dimension

# **Confidence Threshold**
CONFIDENCE_THRESHOLD = 75  

# **Process Images in the Input Folder**
for subdir, _, files in os.walk(input_folder):
    for filename in files:
        img_path = os.path.join(subdir, filename)

        # **Skip Non-Image Files**
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Skipping {filename} (Could not read image)")
            continue

        h, w, _ = img.shape

        # **Detect face mesh using MediaPipe**
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_img)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # **Extract Eye Landmarks**
                left_eye_top = face_landmarks.landmark[159]
                left_eye_bottom = face_landmarks.landmark[145]
                right_eye_top = face_landmarks.landmark[386]
                right_eye_bottom = face_landmarks.landmark[374]

                left_eye_opening = int((left_eye_bottom.y - left_eye_top.y) * h)
                right_eye_opening = int((right_eye_bottom.y - right_eye_top.y) * h)

                avg_eye_opening = (left_eye_opening + right_eye_opening) / 2

                # **Extract Mouth Landmarks**
                upper_lip_top = face_landmarks.landmark[13]
                lower_lip_bottom = face_landmarks.landmark[14]

                mouth_opening = int((lower_lip_bottom.y - upper_lip_top.y) * h)

                # **Preprocess Image & Predict Using Model**
                img_array = preprocess_image(img)
                prediction = model.predict(img_array)  # Model prediction
                class_id = np.argmax(prediction)  # Get class index
                confidence = round(max(prediction[0]) * 100, 2)  # Get highest probability

                # **Classification Logic Based on Model Prediction**
                if confidence >= CONFIDENCE_THRESHOLD:
                    class_label = CLASS_LABELS[class_id]
                    target_folder = class_folders[class_label]
                    print(f"✅ {filename} → {class_label} (Confidence: {confidence}%)")
                    shutil.copy2(img_path, os.path.join(target_folder, filename))
                else:
                    print(f"⚠️ Uncertain: {filename} (Confidence: {confidence}%) - Below threshold")
                    shutil.copy2(img_path, os.path.join(class_folders["Uncertain"], filename))
        else:
            print(f"🚫 No face detected in {filename}. Marking as uncertain.")
            shutil.copy2(img_path, os.path.join(class_folders["Uncertain"], filename))

print("\n✅ Classification completed! High-confidence images sorted into respective classes.")


✅ Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
✅ Drowsy (1).png → Yawning (Confidence: 513.12%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
✅ Drowsy (10).png → Yawning (Confidence: 530.94%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
✅ Drowsy (1000).jpg → Yawning (Confidence: 838.3%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
✅ Drowsy (1002).jpg → Yawning (Confidence: 728.13%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
✅ Drowsy (1006).jpg → Yawning (Confidence: 746.18%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
✅ Drowsy (1010).jpg → Yawning (Confidence: 719.31%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
✅ Drowsy (1015).jpg → Yawning (Confidence: 657.29%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
✅ Drowsy (1019).jpg → Yawning (Confidence: 637.72%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
✅ Drowsy (1020).jpg → Yawning (Confidence: 510.09%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
✅ Drowsy (1021).jpg → Yawning (Confidence: 640.13%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
✅ Drowsy (1024).jpg → Yawning (Confiden

KeyboardInterrupt: 

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import shutil
import mediapipe as mp

# **Load MediaPipe Face Mesh**
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

# **Load Pretrained Model**
model = model  # Ensure the path is correct
print("✅ Model loaded successfully!")

# **Get Input Folder from User**
input_folder = r"D:\Major Project\Final Proper\data_need_to_split\American"# Update the input directory

# **Check if the directory exists**
if not os.path.exists(input_folder):
    print(f"❌ Error: The directory '{input_folder}' does not exist.")
    exit()

# **Define Output Base Folder**
output_base = os.path.join(input_folder, "Dataset_Classified")

# **Class Labels Mapping**
CLASS_LABELS = {
    0: "Eyes Closed",   # Drowsy
    1: "Happy",         # Non-Drowsy
    2: "Neutral",       # Non-Drowsy
    3: "Yawning"        # Drowsy
}

# **Create Output Folders Dynamically**
os.makedirs(output_base, exist_ok=True)
for label in CLASS_LABELS.values():
    os.makedirs(os.path.join(output_base, label), exist_ok=True)

# **Create 'uncertain' folder**
uncertain_folder = os.path.join(output_base, "Uncertain")
os.makedirs(uncertain_folder, exist_ok=True)

# **Preprocessing Function**
def preprocess_image(img, target_size=(224, 224)):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize
    return np.expand_dims(img, axis=0)  # Add batch dimension

# **Confidence Threshold**
CONFIDENCE_THRESHOLD = 75  

# **Process Images in the Input Folder**
for subdir, _, files in os.walk(input_folder):
    for filename in files:
        img_path = os.path.join(subdir, filename)

        # **Skip Non-Image Files**
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Skipping {filename} (Could not read image)")
            continue

        # **Preprocess Image & Predict Using Model**
        img_array = preprocess_image(img)
        prediction = model.predict(img_array)  # Model prediction
        class_id = np.argmax(prediction)  # Get class index
        confidence = round(max(prediction[0]) * 100, 2)  # Get highest probability

        # **Get Class Label and Target Folder**
        class_label = CLASS_LABELS.get(class_id, "Unknown")
        target_folder = os.path.join(output_base, class_label)

        # **Sort Based on Confidence Level**
        if confidence >= CONFIDENCE_THRESHOLD:
            print(f"✅ {filename} → {class_label} (Confidence: {confidence}%)")
            shutil.copy2(img_path, os.path.join(target_folder, filename))
        else:
            print(f"⚠️ Uncertain: {filename} (Confidence: {confidence}%) - Below threshold")
            shutil.copy2(img_path, os.path.join(uncertain_folder, filename))

print("\n✅ Classification completed! High-confidence images sorted, uncertain ones stored separately.")


✅ 2525.jpg → Yawning (Confidence: 278.72%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
✅ 2526.jpg → Yawning (Confidence: 293.62%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
✅ 2527.jpg → Yawning (Confidence: 260.18%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
✅ 2528.jpg → Yawning (Confidence: 240.39%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
✅ 2529.jpg → Yawning (Confidence: 294.86%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
✅ 253.jpg → Happy (Confidence: 230.39%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
✅ 2530.jpg → Yawning (Confidence: 361.24%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
✅ 2531.jpg → Happy (Confidence: 265.71%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
✅ 2532.jpg → Yawning (Confidence: 321.28%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
✅ 2533.jpg → Yawning (Confidence: 272.18%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
✅ 2534.jpg → Yawning (Confidence: 396.22%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
✅ 2535.jpg → Yawning (Confidence: 368.38%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
✅ 2536.jpg → Yawning (Confidence: